In [87]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [88]:
premier_url = 'https://fbref.com/en/comps/9/wages/Premier-League-Wages'
data = requests.get(premier_url)

In [89]:
soup = BeautifulSoup(data.text, 'html.parser')

In [90]:
salary_table_html = soup.select('table.stats_table')[1]

In [91]:
df = pd.read_html(str(salary_table_html))[0]

C:\Users\matas\AppData\Local\Temp\ipykernel_20736\1517590594.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(salary_table_html))[0]


In [92]:
df.head()

,Rk,Player,Nation,Pos,Squad,Age,Weekly Wages,Annual Wages,Notes
0,1,Kevin De Bruyne,be BEL,MF,Manchester City,32,"£ 400,000 (€ 468,172, $511,381)","£ 20,800,000 (€ 24,344,950, $26,591,812)",NaN
1,2,Erling Haaland,no NOR,FW,Manchester City,23,"£ 375,000 (€ 438,911, $479,420)","£ 19,500,000 (€ 22,823,391, $24,929,824)",NaN
2,3,Casemiro,br BRA,MF,Manchester Utd,31,"£ 350,000 (€ 409,651, $447,458)","£ 18,200,000 (€ 21,301,831, $23,267,836)",NaN
3,4,Mohamed Salah,eg EGY,FW,Liverpool,31,"£ 350,000 (€ 409,651, $447,458)","£ 18,200,000 (€ 21,301,831, $23,267,836)",NaN
4,5,Raphaël Varane,fr FRA,DF,Manchester Utd,30,"£ 340,000 (€ 397,946, $434,674)","£ 17,680,000 (€ 20,693,208, $22,603,040)",NaN


In [93]:
df['Nation'].unique()

array(['be BEL', 'no NOR', 'br BRA', 'eg EGY', 'fr FRA', 'eng ENG',
       'pt POR', 'de GER', 'nl NED', 'es ESP', 'gh GHA', 'hr CRO',
       'kr KOR', 'ch SUI', 'ar ARG', 'ua UKR', 'dk DEN', 'ec ECU',
       'uy URU', 'ml MLI', 'cm CMR', 'se SWE', 'hu HUN', 'sct SCO',
       'it ITA', 'mx MEX', 'jm JAM', nan, 'cz CZE', 'ro ROU', 'wls WAL',
       'pl POL', 'jp JPN', 'sn SEN', 'nz NZL', 'ng NGA', 'bf BFA',
       'gr GRE', 'ci CIV', 'co COL', 'nir NIR', 'ma MAR', 'rs SRB',
       'il ISR', 'us USA', 'py PAR', 'dz ALG', 'ie IRL', 'gw GNB',
       'ga GAB', 'sk SVK', 'al ALB', 'tr TUR', 'cl CHI', 'at AUT',
       'is ISL', 'za RSA', 'ba BIH', 'zw ZIM', 'xk KVX', 'cd COD',
       'ca CAN', 'ir IRN', 'gd GRN', 'ee EST', 'mk MKD', 'tg TOG',
       'au AUS'], dtype=object)

In [94]:
from io import StringIO

def fetch_season_data(season):
    # Generate the URL for a given season
    url = f'https://fbref.com/en/comps/9/{season}/wages/{season}-Premier-League-Wages'
    data = requests.get(url)
    
    # Check if the request was successful
    if data.status_code == 200:
        soup = BeautifulSoup(data.text, 'html.parser')
        # Check if the expected table is found
        if soup.select('table.stats_table'):
            salary_table_html = soup.select('table.stats_table')[1]
            # Pandas read_html now takes an IO stream, so we use StringIO here
            df = pd.read_html(StringIO(salary_table_html.prettify()))[0]
            df['Season'] = season  # Add a season column
            return df
        else:
            print(f'Table for season {season} not found on page.')
            return pd.DataFrame()  # Return an empty DataFrame if table is not found
    else:
        print(f'Data for season {season} not found with status code {data.status_code}.')
        return pd.DataFrame()  # Return an empty DataFrame if data is not found
    all_seasons_df = pd.DataFrame()

# Loop through the seasons from 2023/24 to 2006/07
for end_year in range(2024, 2006, -1):  # Change to 2025 if you need the 2024/2025 season
    # Convert end year to season string
    season = f'{end_year-1}-{end_year}'
    # Fetch and append the season data
    season_df = fetch_season_data(season)
    all_seasons_df = pd.concat([all_seasons_df, season_df], ignore_index=True)

# Check the combined DataFrame
print(all_seasons_df.head())

# Export to CSV if needed
 all_seasons_df.to_csv('premier_league_wages_2006_2023.csv', index=False)


IndexError: list index out of range

In [96]:
all_seasons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19443 entries, 0 to 19442
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Rk            19443 non-null  int64 
 1   Player        19443 non-null  object
 2   Nation        19146 non-null  object
 3   Pos           18321 non-null  object
 4   Squad         19443 non-null  object
 5   Age           19443 non-null  int64 
 6   Weekly Wages  19443 non-null  object
 7   Annual Wages  19443 non-null  object
 8   Notes         14637 non-null  object
 9   Season        19443 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.5+ MB


In [97]:
 all_seasons_df.to_csv('premier_league_wages_2006_2023.csv', index=False)